In [1]:
import numpy as np
from csc.utils import *
from csc.atoms import *
from csc.workbench import *
from csc.dictionary import *
from itertools import product
import matplotlib.pyplot as plt

# Atom parameters
bmin = 0.005
bmax = 0.06
b_step = 0.005
b_values = np.arange(bmin, bmax, b_step)

ymin = 0.010
ymax = 0.036
y_step = 0.002
y_values = np.arange(ymin, ymax, y_step)

s_values = [1]

dictionary = ZSDictionary.from_values(b_values, y_values, s_values)
print('The dictionary has {} different atoms'.format(len(dictionary)))

The dictionary has 143 different atoms


### Workbench initialization with a database path

In [2]:
batchSize = 200

pomme = False

if pomme :
    constrained_signals_db = f'/home/gomes/synthetic-data/constrained-signals-{batchSize}.json'

    borelli_mmp81_db = f'/home/gomes/synthetic-data/borelli-mmpdf81-{batchSize}.json'
    borelli_mmp27_db = f'/home/gomes/synthetic-data/borelli-mmpdf27-{batchSize}.json'
    borelli_mmp9_db = f'/home/gomes/synthetic-data/borelli-mmpdf9-{batchSize}.json'
    borelli_mmp3_db = f'/home/gomes/synthetic-data/borelli-mmpdf3-{batchSize}.json'

    icassp_mp_db = f'/home/gomes/synthetic-data/icassp-mp-{batchSize}.json'
    icassp_omp_db = f'/home/gomes/synthetic-data/icassp-omp-{batchSize}.json'
    icassp_alphaCSC_db = f'/home/gomes/synthetic-data/icassp-alphaCSC-{batchSize}.json'

else:
    constrained_signals_db = f'C:\\Users\\gomes\\Documents\\ENS\\00_M1\\00_BORELLI\\01_WorkInProgress\\\synthetic-data\\constrained-signals-{batchSize}.json'

    borelli_mmp81_db = f'C:\\Users\\gomes\\Documents\\ENS\\00_M1\\00_BORELLI\\01_WorkInProgress\\synthetic-data\\borelli-mmpdf81-{batchSize}.json'
    borelli_mmp27_db = f'C:\\Users\\gomes\\Documents\\ENS\\00_M1\\00_BORELLI\\01_WorkInProgress\\synthetic-data\\borelli-mmpdf27-{batchSize}.json'
    borelli_mmp9_db = f'C:\\Users\\gomes\\Documents\\ENS\\00_M1\\00_BORELLI\\01_WorkInProgress\\synthetic-data\\borelli-mmpdf9-{batchSize}.json'
    borelli_mmp3_db = f'C:\\Users\\gomes\\Documents\\ENS\\00_M1\\00_BORELLI\\01_WorkInProgress\\synthetic-data\\borelli-mmpdf3-{batchSize}.json'
    
    icassp_mp_db = f'C:\\Users\\gomes\\Documents\\ENS\\00_M1\\00_BORELLI\\01_WorkInProgress\\synthetic-data\\icassp-mp-{batchSize}.json'
    icassp_omp_db = f'C:\\Users\\gomes\\Documents\\ENS\\00_M1\\00_BORELLI\\01_WorkInProgress\\synthetic-data\\icassp-omp-{batchSize}.json'
    icassp_alphaCSC_db = f'C:\\Users\\gomes\\Documents\\ENS\\00_M1\\00_BORELLI\\01_WorkInProgress\\synthetic-data\\icassp-alphaCSC-{batchSize}.json'

In [3]:
workbench = CSCWorkbench(constrained_signals_db)
workbench.set_dictionary(dictionary)
workbench.load_data()

In [4]:
CONNECTIONS = 3
NB_BRANCHES = 9
DISSIMILARITY = 0.4

#### Signal ID = 1987

In [5]:
signal_id = 1987
signal_dict = workbench.signalDictFromId(signal_id)
noisy_signal = signal_dict['signal']
true_atoms = signal_dict['atoms']
signal_sparsity = len(true_atoms)
true_signal = np.zeros_like(noisy_signal)
print(true_atoms)
for atom_dict in true_atoms:
    zs_atom = ZSAtom.from_dict(atom_dict)
    zs_atom.padBothSides(dictionary.getAtomsLength())
    atom_signal = zs_atom.getAtomInSignal(signal_length=len(noisy_signal), offset=zs_atom['x'])
    true_signal += atom_signal

[{'x': 217, 'b': 0.055, 'y': 0.012, 's': 1.653959710339202e-05}, {'x': 123, 'b': 0.04, 'y': 0.01, 's': 1.2558767847718308e-05}, {'x': 30, 'b': 0.034999999999999996, 'y': 0.014, 's': 2.0432484032758824e-05}]


TypeError: unsupported operand type(s) for +: 'NoneType' and 'int'

In [ ]:
mmp_tree = MMPTree(dictionary=dictionary, signal=noisy_signal, sparsity=signal_sparsity, connections_level=CONNECTIONS, dissimilarity=DISSIMILARITY)
mmp_tree.runMMPDF(branches=NB_BRANCHES, verbose=True)
approx, infos = mmp_tree.getResult()